# Creating a new model based on a dataset of 4028 images. Yolo V11

#Before running this notebook make sure to download your images with annotations, pre-labeled from Roboflow either in the format of YOLOv8 or YOLOv11.

# You'll then upload this Dataset to google colab or if you're working locally make sure you have a home directory defined


## Mount your google drive to access the dataset you uploaded from your computer that you got from roboflow


In [1]:
#move all 6 datasets into proper directories in google drive
from google.colab import drive #mount the drive to access files on google
drive.mount('/content/drive')

Mounted at /content/drive


### you'll need to run the next line sometimes after having to restart the kernel. please skip if this is not the case

In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


### need to declare this so that you can run Gradio on Google Colab

In [ ]:
import os
os.environ["LC_ALL"] = "C.UTF-8"
os.environ["LANG"] = "C.UTF-8"


## creating a data.yaml file for your dataset in your defined directory. Your model will reference your YAML File to run the model. In these next lines we also relabel our dataset. When pulling from roboflow you'll get dataset with classes that are under represented so you need to re label to the classes you actually want to train the model on. not only that some classes will be labeled differently, but still represent the same thing i.e. Bike, Cyclist, bicycle, bike. These are all the same but we want the model to treat these as the same class

### TL:DR sometimes you need to relabel your dataset. You only need to run through these lines once, they will be declared in your directory after.



In [ ]:
#checking for classes in the dataset to identify them for the yaml file
import os

labels_dir = "/content/drive/MyDrive/Carbike detection.v1i.yolov11/train/labels"

# Get all label files
label_files = [os.path.join(labels_dir, f) for f in os.listdir(labels_dir) if f.endswith(".txt")]

# Extract class IDs from the first few label files
class_ids = set()
for label_file in label_files:  # remove [:50] to check all
    with open(label_file, "r") as f:
        for line in f:
            class_id = int(line.split()[0])
            class_ids.add(class_id)

print(f"Number of classes: {len(class_ids)}")
print(f"Class IDs: {class_ids}")


Number of classes: 11
Class IDs: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


### Have to check these mappings so that we can relabel classes

In [ ]:
import yaml

yaml_path = "/content/drive/MyDrive/Carbike detection.v1i.yolov11/data.yaml"

with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

class_names = data['names']
print("Class names dictionary:", class_names)

# Optional: print class names for your found IDs
class_ids ={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
for cid in class_ids:
    name = class_names[cid] if cid < len(class_names) else "UNKNOWN"
    print(f"Class {cid}: {name}")


Class names dictionary: ['Bicycle', 'Car', 'bicycle', 'bus', 'car', 'cyclist', 'heavy_truck', 'light_truck', 'truck', 'van', 'vehicle']
Class 0: Bicycle
Class 1: Car
Class 2: bicycle
Class 3: bus
Class 4: car
Class 5: cyclist
Class 6: heavy_truck
Class 7: light_truck
Class 8: truck
Class 9: van
Class 10: vehicle


In [ ]:
import yaml

yaml_path = "/content/drive/MyDrive/Carbike detection.v1i.yolov11/data.yaml"
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

class_names = data['names']

# Show all classes defined in the YAML
for i, name in enumerate(class_names):
    print(f"{i}: {name}")


0: Bicycle
1: Car
2: bicycle
3: bus
4: car
5: cyclist
6: heavy_truck
7: light_truck
8: truck
9: van
10: vehicle


## Relabel classes to three: Car, Bike and Other


In [ ]:
car_ids = [1, 3, 4, 6, 7, 8, 9, 10]        # original IDs representing cars
cyclist_ids = [0, 2, 5]     # original IDs representing cyclists

# New mapping: 0=Car, 1=Cyclist
def remap_class(class_id):
    if class_id in car_ids:
        return 0
    elif class_id in cyclist_ids:
        return 1

In [ ]:
import os

labels_dirs = [
    "/content/drive/MyDrive/Carbike detection.v1i.yolov11/test/labels",
    "/content/drive/MyDrive/Carbike detection.v1i.yolov11/valid/labels",
    "/content/drive/MyDrive/Carbike detection.v1i.yolov11/train/labels"
]

for labels_dir in labels_dirs:
    for filename in os.listdir(labels_dir):
        if not filename.endswith(".txt"):
            continue

        file_path = os.path.join(labels_dir, filename)
        new_lines = []

        with open(file_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                class_id = int(parts[0])
                new_class = remap_class(class_id)
                parts[0] = str(new_class)
                new_lines.append(" ".join(parts))

        with open(file_path, "w") as f:
            f.write("\n".join(new_lines))


In [ ]:
#checking for classes in the dataset to identify them for the yaml file
import os

labels_dir = "/content/drive/MyDrive/Carbike detection.v1i.yolov11/train/labels"

# Get all label files
label_files = [os.path.join(labels_dir, f) for f in os.listdir(labels_dir) if f.endswith(".txt")]

# Extract class IDs from the first few label files
class_ids = set()
for label_file in label_files:  # remove [:50] to check all
    with open(label_file, "r") as f:
        for line in f:
            class_id = int(line.split()[0])
            class_ids.add(class_id)

print(f"Number of classes: {len(class_ids)}")
print(f"Class IDs: {class_ids}")

Number of classes: 2
Class IDs: {0, 1}


In [ ]:
# Create the YAML content
yaml_content = """train: /content/drive/MyDrive/Carbike detection.v1i.yolov11/train/images
val: /content/drive/MyDrive/Carbike detection.v1i.yolov11/valid/images

nc: 2
names:
  0: car
  1: cyclist
"""

# Save the YAML file
yaml_path = "/content/drive/MyDrive/Carbike detection.v1i.yolov11/data.yaml"
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print(f"data.yaml created at {yaml_path}")



data.yaml created at /content/drive/MyDrive/Carbike detection.v1i.yolov11/data.yaml


In [ ]:
with open(yaml_path, "r") as f:
    print(f.read())

train: /content/drive/MyDrive/Carbike detection.v1i.yolov11/train/images
val: /content/drive/MyDrive/Carbike detection.v1i.yolov11/valid/images

nc: 2
names:
  0: car
  1: cyclist



# "This is where the fun begins". Now we train the model using the small size V8 model

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.8/974.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

### next two blocks use to double check to make sure you have the yolo model you want to run. download this https://docs.ultralytics.com/tasks/detect/#models:~:text=Models%20download%20automatically%20from%20the%20latest%20Ultralytics%20release%20on%20first%20use.

### really its up to you what size model you want to use, nano will be the fastest training, with lowest accuracy, while x will be the slowest training, but highest accuracy I did medium here

In [ ]:
import os

model_path = "/content/drive/MyDrive/Carbike detection.v1i.yolov11/yolo11m.pt"
if os.path.exists(model_path):
    print("Model file found!")
else:
    print("Model file NOT found. Downloading now...")


Model file found!


In [ ]:
from ultralytics import YOLO

model_path = "/content/drive/MyDrive/Carbike detection.v1i.yolov11/yolo11m.pt"

try:
    model = YOLO(model_path)  # Explicitly use medium model
except FileNotFoundError:
    print("Error: yolo11m.pt not found! Please download it manually.")
    raise

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### run !nvidia-smi to see your cuda version, CPU and GPU performance. this doesn't matter as much if you're running this on google colab, but locally it is very important

In [ ]:
!nvidia-smi


Tue Apr  8 17:10:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# this setup for the model is bare bones, but it gets the job done, I wouldn't recommend getting too much in the weeds with hyperparameters, but if you're interested visit https://docs.ultralytics.com/modes/train/#resuming-interrupted-trainings:~:text=a%20training%20run.-,Train%20Settings,tuning%20and%20experimentation%20with%20these%20settings%20are%20crucial%20for%20optimizing%20performance.,-Argument


In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO('/content/drive/MyDrive/Carbike detection.v1i.yolov11/yolo11m.pt')

# Train the model
model.train(
    data='/content/drive/MyDrive/Carbike detection.v1i.yolov11/data.yaml',
    epochs=40,
    imgsz=768, # bump it up in terms of
    batch=16, # bump up batch size
    save=True,
    name='Car and Bike detections',
    project='/content/drive/MyDrive/Carbike detection.v1i.yolov11/runs'
)

Ultralytics 8.3.104 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/Carbike detection.v1i.yolov11/yolo11m.pt, data=/content/drive/MyDrive/Carbike detection.v1i.yolov11/data.yaml, epochs=40, time=None, patience=100, batch=16, imgsz=768, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Carbike detection.v1i.yolov11/runs, name=Car and Bike detections, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=No

100%|██████████| 755k/755k [00:00<00:00, 46.5MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 118MB/s]


AMP: checks passed ✅


...

                   all        807       3825      0.808      0.768      0.797      0.566



40 epochs completed in 1.829 hours.
Optimizer stripped from /content/drive/MyDrive/Carbike detection.v1i.yolov11/runs/Car and Bike detections/weights/last.pt, 40.5MB
Optimizer stripped from /content/drive/MyDrive/Carbike detection.v1i.yolov11/runs/Car and Bike detections/weights/best.pt, 40.5MB

Validating /content/drive/MyDrive/Carbike detection.v1i.yolov11/runs/Car and Bike detections/weights/best.pt...
Ultralytics 8.3.104 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:23<00:00,  1.11it/s]


                   all        807       3825      0.796      0.768       0.81       0.57
                   car        616       3266      0.867      0.793      0.855      0.634
               cyclist        343        559      0.726      0.742      0.765      0.505
Speed: 0.3ms preprocess, 13.5ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to /content/drive/MyDrive/Carbike detection.v1i.yolov11/runs/Car and Bike detections


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7dcb16a360d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

### define model path and run validation

In [3]:
from ultralytics import YOLO

# Load the saved model
model_path = '/content/drive/MyDrive/Carbike detection.v1i.yolov11/runs/Car and Bike detections/weights/best.pt'
model = YOLO(model_path)

print(f"Model loaded from: {model_path}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Model loaded from: /content/drive/MyDrive/Carbike detection.v1i.yolov11/runs/Car and Bike detections/weights/best.pt


In [4]:
# Validate the model
metrics = model.val()

# Print validation results
print(metrics)

Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 25.2MB/s]
val: Scanning /content/drive/MyDrive/Carbike detection.v1i.yolov11/valid/labels.cache... 807 images, 0 backgrounds, 0 corrupt: 100%|██████████| 807/807 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 21, len(boxes) = 3825. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 51/51 [00:49<00:00,  1.03it/s]


                   all        807       3825      0.796       0.77       0.81      0.571
                   car        616       3266      0.866      0.793      0.854      0.635
               cyclist        343        559      0.727      0.747      0.765      0.508
Speed: 0.3ms preprocess, 28.1ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/val
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d323c13db50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0

## realtime inference and csv extraction

### to do real-time inference we leverage Gradio and a github library for IDing individual objects in Computer vision settings


In [5]:
!pip install -q ultralytics gradio opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.7 MB/s eta 0:00:00


## using ID to track individual objects across frames


### run the next 6 blocks everytime


In [6]:
import os
import shutil
folder_path = '/content/sort'
if os.path.exists(folder_path):
    shutil.rmtree(folder_path, ignore_errors=True)
    print("Folder deleted successfully.")
else:
    print("Folder does not exist.")



Folder does not exist.


In [7]:
!pip install filterpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=3b91c5972f8c0dd5bdaa612012735c04b4a3ea824754330a04c388f9edf01f1a
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy


In [8]:
!git clone https://github.com/abewley/sort.git

Cloning into 'sort'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 208 (delta 2), reused 1 (delta 1), pack-reused 203 (from 2)
Receiving objects: 100% (208/208), 1.21 MiB | 2.68 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [9]:
%cd /content/sort
!mv sort.py sort_module.py


/content/sort


In [10]:
# Replace 'TkAgg' with 'Agg' in the source code
!sed -i 's/TkAgg/Agg/g' /content/sort/sort_module.py


In [11]:
import sys
sys.path.append("/content/sort")
from sort_module import Sort



## Gradio Attempts

In [ ]:
#multiple streets
import cv2
import time
import threading
import numpy as np
import pandas as pd
import gradio as gr
from datetime import datetime, timedelta
import os
from ultralytics import YOLO
import matplotlib.pyplot as plt
import io
from PIL import Image

import sys
sys.path.append("/content/sort")
from sort_module import Sort

# Load YOLOv11 model
model = YOLO("/content/drive/MyDrive/Carbike detection.v1i.yolov11/runs/Car and Bike detections/weights/best.pt")

# Define multiple camera streams
streams = {
    "BroadatArthur": "https://media-sfs6.vdotcameras.com/rtplive/richmondstream006/chunklist_w1538414768.m3u8",
    "BroadatSecond": "https://media-sfs8.vdotcameras.com/rtplive/richmondstream017/chunklist_w1556801349.m3u8",
    "JacksonatThird": "https://media-sfs4.vdotcameras.com/rtplive/richmondstream012/chunklist_w866713979.m3u8",
    #"BroadatFourth": "https://media-sfs3.vdotcameras.com/rtplive/richmondstream003/chunklist_w484278230.m3u8",
    "GraceatNinth": "https://media-sfs1.vdotcameras.com/rtplive/richmondstream009/chunklist_w1745749359.m3u8",
    "BroadatEighth": "https://media-sfs2.vdotcameras.com/rtplive/richmondstream002/chunklist_w144674066.m3u8"
}

latest_frames = {cam: np.zeros((480, 854, 3), dtype=np.uint8) for cam in streams}
trackers = {cam: Sort() for cam in streams}
csv_filenames = {cam: f"car_bike_counts_{cam.lower()}.csv" for cam in streams}

# Initialize CSVs
for cam, filename in csv_filenames.items():
    if not os.path.exists(filename):
        pd.DataFrame(columns=["timestamp", "unique_car_ids", "unique_bike_ids"]).to_csv(filename, index=False)

def run_inference(cam_name, stream_url):
    cap = cv2.VideoCapture(stream_url)
    if not cap.isOpened():
        print(f"❌ Error: Cannot open stream for {cam_name}")
        return

    tracker = trackers[cam_name]
    csv_file = csv_filenames[cam_name]
    unique_car_ids = set()
    unique_bike_ids = set()
    last_logged_time = time.time()

    while True:
        try:
            ret, frame = cap.read()
            if not ret:
                continue

            frame = cv2.resize(frame, (854, 480))
            results = model(frame, verbose=False)[0]

            dets = []
            for box in results.boxes:
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                x1, y1, x2, y2 = box.xyxy[0]
                dets.append([x1.item(), y1.item(), x2.item(), y2.item(), conf])
            dets_np = np.array(dets)

            try:
                tracked_objects = tracker.update(dets_np)
            except Exception as e:
                print(f"[SORT ERROR] {e}")
                continue

            for i, track in enumerate(tracked_objects):
                if i < len(results.boxes):
                    cls_id = int(results.boxes[i].cls[0])
                    cls_name = model.names[cls_id].lower()
                    track_id = int(track[4])
                    if "car" in cls_name:
                        unique_car_ids.add(track_id)
                    elif "bike" in cls_name or "cyclist" in cls_name:
                        unique_bike_ids.add(track_id)

            frame = results.plot()
            latest_frames[cam_name] = frame

            current_time = time.time()
            if current_time - last_logged_time >= 60:
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                row = {
                    "timestamp": timestamp,
                    "unique_car_ids": len(unique_car_ids),
                    "unique_bike_ids": len(unique_bike_ids)
                }
                pd.DataFrame([row]).to_csv(csv_file, mode="a", header=False, index=False)
                print(f"✅ {cam_name} Logged: {row}")
                unique_car_ids.clear()
                unique_bike_ids.clear()
                last_logged_time = current_time

            time.sleep(0.05)

        except Exception as e:
            print(f"[ERROR] {cam_name} crashed: {e}")
            continue

# Launch tracking threads
for cam, url in streams.items():
    threading.Thread(target=run_inference, args=(cam, url), daemon=True).start()

def generate_plot(cam_name):
    try:
        df = pd.read_csv(csv_filenames[cam_name])
        if df.empty:
            return np.zeros((400, 800, 3), dtype=np.uint8)

        df['timestamp'] = pd.to_datetime(df['timestamp'])
        cutoff = datetime.now() - timedelta(minutes=60)
        df = df[df['timestamp'] >= cutoff]
        df = df.groupby(pd.Grouper(key='timestamp', freq='1min')).sum().reset_index()

        plt.figure(figsize=(8, 4))
        plt.plot(df['timestamp'], df['unique_car_ids'], label='Cars', marker='o')
        plt.plot(df['timestamp'], df['unique_bike_ids'], label='Bikes', marker='x')
        plt.xticks(rotation=45)
        plt.title(f"{cam_name} Count Changes (Last 60 Min)")
        plt.xlabel("Time")
        plt.ylabel("Count")
        plt.legend()
        plt.tight_layout()

        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        plt.close()
        buf.seek(0)
        img = Image.open(buf)
        return np.array(img)

    except Exception as e:
        print(f"[PLOT ERROR - {cam_name}] {e}")
        return np.zeros((400, 800, 3), dtype=np.uint8)

with gr.Blocks() as demo:
    gr.Markdown("# 🧭 Multi-Camera Traffic Dashboard")

    cam_dropdown = gr.Dropdown(choices=["All"] + list(streams.keys()), label="Select Camera", value="All")
    refresh_btn = gr.Button("🔁 Refresh All")

    stream_boxes = {}

    with gr.Row():
        for cam in streams:
            with gr.Column(visible=True) as cam_box:
                stream_img = gr.Image(type="numpy", label=f"{cam} Stream")
                chart_img = gr.Image(type="numpy", label=f"{cam} Chart")
                stream_boxes[cam] = (cam_box, stream_img, chart_img)

    def refresh_all(selected):
        visible = selected == "All"
        updates = {}
        for cam, (box, stream_out, chart_out) in stream_boxes.items():
            show = visible or cam == selected
            frame = latest_frames[cam]
            chart = generate_plot(cam)
            updates[box] = gr.update(visible=show)
            updates[stream_out] = gr.update(value=frame)
            updates[chart_out] = gr.update(value=chart)
        return updates

    refresh_btn.click(fn=refresh_all, inputs=[cam_dropdown], outputs=[c for triple in stream_boxes.values() for c in triple])

    # Trigger initial refresh
    demo.load(fn=refresh_all, inputs=[cam_dropdown], outputs=[c for triple in stream_boxes.values() for c in triple])

demo.launch(debug=True, share=True)


Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
* Running on public URL: https://e81ca1dd27166bc0e9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
YOLO11m summary (fused): 125 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs
[SORT ERROR] operands could not be broadcast together with shapes (0,) (1,2) 
[SORT ERROR] operands could not be broadcast together with shapes (0,) (1,2) 
✅ BroadatArthur Logged: {'timestamp': '2025-04-16 21:52:14', 'unique_car_ids': 56, 'unique_bike_ids': 2}
✅ JacksonatThird Logged: {'timestamp': '2025-04-16 21:52:15', 'unique_car_ids': 20, 'unique_bike_ids': 0}
✅ BroadatFourth Logged: {'timestamp': '2